In [7]:
import cv2
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import json

In [39]:
image_directory  = "/workspace/data/aviodrome/plane/small_input/mask/raw"
combine_directory  = "/workspace/data/aviodrome/plane/small_input/mask/out"

In [31]:
with open(f"{image_directory}/annotations.json",encoding="utf-8") as f:
    annotations = json.load(f)

In [45]:
import json
import os
import shutil

# Define your paths
input_dir = '/workspace/data/aviodrome/plane/small_input/images'
masks_dir = '/workspace/data/aviodrome/plane/small_input/mask/out'
output_dir = '/workspace/data/aviodrome/plane/small_input/mask/renamed'
json_file_path = '/workspace/data/aviodrome/plane/small_input/annotations.json'

# Load the JSON data
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Loop through each item in the JSON data
for item in data:
    # Extract the relevant fields from the JSON
    image_path = item['image']
    # Extract the actual filename, removing any prefix and keeping only 'frame_xxxx.png'
    image_filename = os.path.basename(image_path).split('-')[-1]  # This should give us 'frame_xxxx.png'
    mask_id = item['id']

    # Find the corresponding mask file
    mask_files = [f for f in os.listdir(masks_dir) if f.startswith(f'task-{mask_id}-annotation')]

    # Copy each mask to the output directory with the new filename
    for mask_file in mask_files:
        source_mask_path = os.path.join(masks_dir, mask_file)
        destination_mask_path = os.path.join(output_dir, image_filename)  # Final name: frame_xxxx.png
        shutil.copy2(source_mask_path, destination_mask_path)

        print(f'Copied {mask_file} to {destination_mask_path}')


Copied task-2-annotation-2-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00019.png
Copied task-45-annotation-45-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00277.png
Copied task-29-annotation-29-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00181.png
Copied task-8-annotation-8-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00055.png
Copied task-26-annotation-26-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00163.png
Copied task-4-annotation-4-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00031.png
Copied task-34-annotation-34-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mask/renamed/frame_00211.png
Copied task-35-annotation-35-by-2-tag-Airplane-0.png to /workspace/data/aviodrome/plane/small_input/mas

In [30]:
annotations[0]["inner_id"]

2

In [43]:
def morph(image):
    kernel = np.ones((5,5), np.uint8)
    close_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel,iterations = 2)
    open_image = cv2.morphologyEx(close_image, cv2.MORPH_OPEN, kernel,iterations = 2)
    dilation = cv2.dilate(open_image,np.ones((7,7), np.uint8),iterations = 1)
    return dilation


# Organize the files by task
task_files = {}
for file_path in glob(os.path.join(image_directory, '*.png')):
    filename = os.path.basename(file_path)
    task_number = filename.split('-')[1]  # Extracts the task number
    task_files.setdefault(task_number, []).append(file_path)

# Process each task
for task_number, file_list in task_files.items():
    # Sort the file list to ensure correct order
    file_list.sort(key=lambda x: int(x.split('-')[-1].split('.')[0]))

    # Initialize a variable for the combined mask
    combined_mask = None

    # Read and merge images
    for file_path in file_list:
        image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if combined_mask is None:
            combined_mask = image
        else:
            combined_mask = cv2.bitwise_or(combined_mask, image)

    processed_mask = morph(combined_mask)

    # Save or display the result
    output_path = os.path.join(combine_directory, f'task-{task_number}-annotation-{task_number}-by-2-tag-Airplane-0.png')
    cv2.imwrite(output_path, processed_mask)
    print(f'Processed mask saved as {output_path}')

Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-1-annotation-1-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-2-annotation-2-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-3-annotation-3-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-4-annotation-4-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-5-annotation-5-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-6-annotation-6-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-7-annotation-7-by-2-tag-Airplane-0.png
Processed mask saved as /workspace/data/aviodrome/plane/small_input/mask/out/task-8-annotation-8-by-2-tag-Airplane-0.png
Processed mask saved as /workspa